In [1]:
# to handle datasets
import pandas as pd
# to make plots
import plotly.express as px
from dash import Dash, dcc, html, Input, Output
# importing helper functions
import helper_functions as hlp
# Modelling
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
# First read dataset
url = "https://statistics.gov.scot/downloads/cube-table?uri=http%3A%2F%2Fstatistics.gov.scot%2Fdata%2Frecorded-crime"
crimes_df = pd.read_csv(url)

In [3]:
# Creating cleaner data frame with crimes with only ratios (not counts) for counties.
# Crime types and date columns have been transformed to ordered categorical type
# Each row contains a unique crime ratio for each county, date and crime type
parsed_data = hlp.parse_df(crimes_df) # This function returns a list with a df and another list
# Pivoting table here so we can have crime types as columns for plotting
crimes_ratios_df = parsed_data[0]
pivoted_crimes_ratios_df = pd.pivot_table(crimes_ratios_df, values="Value", index=["Date","LAD"], columns="Crime").reset_index("LAD")

In [122]:
# Question 1: How do Scottish Crime rates vary over time?
# Plotting overall Scottish crime ratios over time
fig = px.line(data_frame=pivoted_crimes_ratios_df, x=pivoted_crimes_ratios_df.index, y="All Crimes", color="LAD")
fig.show()
fig.write_html("assets/scottish_crime_ratios_over_time.html", default_width="100%", default_height="100%")

In [8]:
# Question 2: Which counties show highest increase and decrease in crime rates, over the past two years?
# Here I will select all crimes and calculate percentage changes between the last two years, for each county
all_crimes_df = crimes_ratios_df[crimes_ratios_df["Crime"]=="All Crimes"]
all_crimes_pivoted_df = pd.pivot_table(all_crimes_df, values="Value", index=["LAD"], columns="Date")
all_crimes_pivoted_df["Percentage Change (2023-2022)"] = ((all_crimes_pivoted_df["2022/2023"] - all_crimes_pivoted_df["2021/2022"])  / all_crimes_pivoted_df["2021/2022"] ) * 100

# Here I am plotting the bar graph of percentage changes
fig = px.bar(data_frame=all_crimes_pivoted_df.sort_values("Percentage Change (2023-2022)", ascending=False).reset_index(),
             x="Percentage Change (2023-2022)", color="LAD",
             color_discrete_sequence=px.colors.qualitative.G10,
             title="Percentage Change in Crime Rates from 2022/2023 to 2021/2022")
fig.show()
fig.write_html("assets/percent_change_crimes.html", default_width="100%", default_height="100%")

In [7]:
crime_subtypes = crimes_ratios_df[~crimes_ratios_df["Crime"].str.contains('Crimes')]

filter_counties = ["East Dunbartonshire", "East Renfrewshire", 
                   "Glasgow City", "Aberdeen City", 
                   "Orkney Islands", "Shetland Islands", 
                   "Inverclyde", "Falkirk"]

crime_subtypes_filtered = crime_subtypes[crime_subtypes["LAD"].isin(filter_counties)]
crime_subtypes_filtered = crime_subtypes_filtered[crime_subtypes_filtered["Date"]=="2022/2023"]
crime_subtypes_filtered.loc[:,"Crime"] = crime_subtypes_filtered.Crime.cat.remove_unused_categories()


ordered_crimes = parsed_data[1][1:]
# Question 3: Which areas have similar crime types?
# Stacked Bar Plot
fig = px.bar(crime_subtypes_filtered, x="LAD",y="Value", color="Crime", 
             category_orders={"Crime": ordered_crimes},
             labels={"Crime Rates": "Crime"},
             title="Distribution of Different Crime Types in Scottish Counties for 2022/2023",
             barmode="stack")

fig.update_xaxes(type='category', categoryorder='category ascending')

# Show the plot
fig.show()
fig.write_html("assets/different_crime_types_2022-2023.html", default_width="100%", default_height="100%")

In [10]:
# Cluster Analysis to group similar counties based on Crime Type Ratios
features = crime_subtypes[crime_subtypes["Date"]=="2022/2023"].pivot_table(index=['LAD'], columns='Crime', values='Value', fill_value=0).reset_index()
X = features.drop(["LAD"], axis=1)
# Standardize features
scaler = StandardScaler()
X_standardized = scaler.fit_transform(X)

# Choosing number of clusters by Elbow method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42, n_init="auto")
    kmeans.fit(X_standardized)
    wcss.append(kmeans.inertia_)

fig = px.line(wcss)
fig.update_layout(
    xaxis_title="Number of Clusters", yaxis_title="WCSS",
    showlegend=False
)
fig.show()


In [11]:

# Apply K-means clustering
kmeans = KMeans(n_clusters=4, random_state=42, n_init="auto")
clusters = kmeans.fit_predict(X_standardized)
clusters +=1

# Add cluster labels to the dataframe
features['Cluster'] = clusters

In [12]:
# Check cluster means and plot
kmeans_mean_cluster = features.drop(columns=["LAD"]).groupby("Cluster").mean()
fig = px.bar(kmeans_mean_cluster, barmode="stack")
fig.update_layout(
    xaxis_title="Cluster", yaxis_title="Mean Crime Rates",
    xaxis = dict(
        tickmode = 'array',
        tickvals = [1, 2, 3, 4],
        ticktext = ['1', '2', '3', '4']
    ))

fig.show()
fig.write_html("assets/cluster_means_2022-2023.html", default_width="100%", default_height="100%")

In [13]:
# Let's Common Assault vs Vandalism
features.loc[:,"Cluster"] = pd.Categorical(features["Cluster"], ordered=True)
fig = px.scatter(features, x="Group 1: Common assault", y="Group 4: Vandalism", color="Cluster", symbol="LAD")
fig.update_layout()
fig.show()
fig.write_html("assets/vandalism_vs_assault_clusters_2022-2023.html", default_width="100%", default_height="100%")

In [131]:
features[features["Cluster"]==4]

Crime,LAD,Coronavirus Restrictions,Group 1: Common assault,Group 1: Death by dangerous driving,Group 1: Domestic Abuse (Scotland) Act 2018,Group 1: Murder and culpable homicide,Group 1: Other non-sexual violence,Group 1: Robbery,Group 1: Serious assault and attempted murder,Group 2: Causing to view sexual activity or images,...,Group 3: Theft of a motor vehicle,Group 4: Fire-raising,Group 4: Reckless conduct,Group 4: Vandalism,Group 5: Drugs - Possession,Group 5: Drugs - Supply,Group 5: Other crimes against society,Group 5: Weapons possession (not used),Group 5: Weapons possession (used),Cluster
5,Clackmannanshire,0,111,0,4,0,7,4,9,3,...,5,4,6,75,38,10,0,9,15,4
6,Dumfries and Galloway,0,102,0,4,0,10,1,4,4,...,5,5,5,76,67,10,0,8,6,4
8,East Ayrshire,0,109,0,3,0,6,3,5,3,...,6,5,7,77,43,9,1,10,7,4
10,East Lothian,0,91,0,2,0,6,1,4,2,...,9,4,5,63,31,5,0,5,3,4
12,Falkirk,0,103,0,3,0,8,3,4,3,...,8,4,7,78,28,7,1,7,12,4
16,Inverclyde,0,83,0,1,0,5,6,9,1,...,8,9,4,69,57,13,0,9,12,4
17,Midlothian,0,96,0,3,0,8,2,5,3,...,10,5,7,71,24,4,0,7,4,4
20,North Ayrshire,0,109,0,3,0,5,3,6,4,...,7,4,5,76,32,5,1,9,9,4
21,North Lanarkshire,0,102,0,2,0,6,2,6,3,...,11,7,7,80,58,6,0,10,14,4
24,Renfrewshire,0,91,0,2,0,5,3,7,3,...,8,6,5,64,46,8,0,7,10,4
